#Implementation model


# Colab with GPU
When running this notebook in colab, ensure that you run it with a GPU as hardware accelerator. To enable this:
- Navigate to Edit → Notebook Settings
- select GPU from the Hardware Accelerator drop-down

With `!nvidia-smi` we can check which GPU was assigned to us in Colab.

In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [2]:
!nvidia-smi

Tue Sep  6 08:46:49 2022       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 460.32.03    Driver Version: 460.32.03    CUDA Version: 11.2     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  Tesla T4            Off  | 00000000:00:04.0 Off |                    0 |
| N/A   58C    P8    11W /  70W |      0MiB / 15109MiB |      0%      Default |
|                               |                      |                  N/A |
+-------------------------------+----------------------+----------------------+
                                                                               
+-------

# Installation
Install EasyNMT by using pip. EasyNMT is using Pytorch. If you have a GPU available on your local machine, have a look at [PyTorch Get Started](https://pytorch.org/get-started/locally/) how to install PyTorch with CUDA support. 

In [3]:
!pip install -U easynmt

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 4.7 MB 37.3 MB/s 
     |████████████████████████████████| 1.3 MB 67.4 MB/s 
     |████████████████████████████████| 68 kB 8.5 MB/s 
     |████████████████████████████████| 6.6 MB 61.3 MB/s 
     |████████████████████████████████| 120 kB 78.8 MB/s 
  Using cached pybind11-2.10.0-py3-none-any.whl (213 kB)
  Created wheel for easynmt: filename=EasyNMT-2.0.2-py3-none-any.whl size=19919 sha256=31b35d4b6ead50783f96775a4e5ff4cbddf88ab0f3fa2a124ca5a0a48dc3b62f
  Stored in directory: /root/.cache/pip/wheels/d1/57/06/53ca38645e14d4537a41e5a36da2026e10f54ae88240dd5190
  Created wheel for fasttext: filename=fasttext-0.9.2-cp37-cp37m-linux_x86_64.whl size=3161724 sha256=82f9245c6b34577e384be7bf277e5df19123df9854f15e7e86d583a91af1b017
  Stored in directory: /root/.cache/pip/wheels/4e/ca/bf/b020d2be95f7641801a6597a29c8f4f19e38f9c02a345bab9b
Successfully built easy

In [4]:
!pip install sacrebleu

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 116 kB 28.8 MB/s 


## Data preview

In [5]:
import pandas as pd
import os
import sys
import sacrebleu

df = pd.read_excel('/content/drive/MyDrive/Health-Claims-NLP/Data/data/EU Health Claims - EN, DE, PL, FR, RO, HU.xlsx')
df


,English,Unnamed: 1,French,Unnamed: 3,German,Unnamed: 5,Polish,Unnamed: 7,Romanian,Unnamed: 9,Hungarian,Unnamed: 11,EC Regulation,Article of EC 1924/2006
0,Nutrient,Claim,Nutrient,Claim,Nutrient,Claim,Nutrient,Claim,Nutrient,Claim,Nutrient,Claim,NaN,NaN
1,Activated charcoal,Activated charcoal contributes to reducing exc...,Charbon actif,Le charbon actif contribue à réduire l’excès d...,Aktivkohle,Aktivkohle trägt zur Verringerung übermäßiger ...,Węgiel aktywowany,Węgiel aktywowany pomaga zmniejszyć występując...,Cărbune activ,Cărbunele activ contribuie la reducerea flatul...,Aktív szén,Az aktív szén részt vesz az étkezést követő tú...,432/2012,Article 13(1): function health claim
2,Alpha-cyclodextrin,Consumption of alpha-cyclodextrin as part of a...,Alpha-cyclodextrine,La consommation d’alpha-cyclodextrine à l’occa...,Alpha-Cyclodextrin,Der Verzehr von Alpha-Cyclodextrin als Bestand...,Alfa-cyklodekstryna,Spożycie alfa-cyklodekstryny w ramach posiłku ...,Alfa-ciclodextrină,Consumul de alfa-ciclodextrină în cadrul unei ...,Alfa-ciklodextrin,Keményítőtartalmú étkezés részeként az alfa-ci...,536/2013,Article 13(1): function health claim
3,Alpha-linolenic acid & linoleic acid,Essential fatty acids are needed for normal gr...,Acide Alpha-linolénique et acide linoléique,Les acides gras essentiels sont nécessaires à ...,Alpha-Linolensäure und Linolsäure,Essenzielle Fettsäuren werden für ein gesundes...,Kwas alfa-linolenowy i kwas linolowy,Niezbędne nienasycone kwasy tłuszczowe są potr...,Acid alfa-linolenic & acid linoleic,Acizii grași esențiali sunt necesari pentru cr...,Alfa-linolénsav és linolsav,Az esszenciális zsírsavakra szükség van a gyer...,983/2009,Article 14(1)(b): health claim referring to ch...
4,Alpha-linolenic acid (ALA),ALA contributes to the maintenance of normal b...,Acide alpha-linolénique (ALA),L’acide alpha-linolénique (ALA) contribue au m...,Alpha-Linolensäure (ALA),ALA trägt zur Aufrechterhaltung eines normalen...,Kwas alfa-linolenowy (ALA),Kwas alfa-linolenowy (ALA) pomaga w utrzymaniu...,Acid alfa-linolenic (ALA),AAL contribuie la menținerea concentrațiilor n...,Alfa-linolénsav (ALA),Az ALA hozzájárul a vér normál koleszterinszin...,432/2012,Article 13(1): function health claim
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
256,Zinc,Zinc contributes to the maintenance of normal ...,Zinc,Le zinc contribue au maintien d’un taux normal...,Zink,Zink trägt zur Erhaltung eines normalen Testos...,Cynk,Cynk pomaga w utrzymaniu prawidłowego poziomu ...,Zinc,Zincul contribuie la menținerea concentrațiilo...,Cink,A cink hozzájárul a vér normál tesztoszteronsz...,432/2012,Article 13(1): function health claim
257,Zinc,Zinc contributes to the maintenance of normal ...,Zinc,Le zinc contribue au maintien d’une vision nor...,Zink,Zink trägt zur Erhaltung normaler Sehkraft bei,Cynk,Cynk pomaga w utrzymaniu prawidłowego widzenia,Zinc,Zincul contribuie la menținerea vederii normale,Cink,A cink hozzájárul a normál látás fenntartásához.,432/2012,Article 13(1): function health claim
258,Zinc,Zinc contributes to the normal function of the...,Zinc,Le zinc contribue au fonctionnement normal du ...,Zink,Zink trägt zu einer normalen Funktion des Immu...,Cynk,Cynk pomaga w prawidłowym funkcjonowaniu układ...,Zinc,Zincul contribuie la funcționarea normală a si...,Cink,A cink hozzájárul az immunrendszer normál műkö...,432/2012,Article 13(1): function health claim
259,Zinc,Zinc contributes to the protection of cells fr...,Zinc,Le zinc contribue à protéger les cellules cont...,Zink,"Zink trägt dazu bei, die Zellen vor oxidativem...",Cynk,Cynk pomaga w ochronie komórek przed stresem o...,Zinc,Zincul contribuie la protejarea celulelor împo...,Cink,A cink hozzájárul a sejtek oxidatív stresszel ...,432/2012,Article 13(1): function health claim


In [6]:
df.iloc[1:,1]


1      Activated charcoal contributes to reducing exc...
2      Consumption of alpha-cyclodextrin as part of a...
3      Essential fatty acids are needed for normal gr...
4      ALA contributes to the maintenance of normal b...
5      Consumption of arabinoxylan as part of a meal ...
                             ...                        
256    Zinc contributes to the maintenance of normal ...
257    Zinc contributes to the maintenance of normal ...
258    Zinc contributes to the normal function of the...
259    Zinc contributes to the protection of cells fr...
260      Zinc has a role in the process of cell division
Name: Unnamed: 1, Length: 260, dtype: object

In [7]:
df.iloc[1:,3]

1      Le charbon actif contribue à réduire l’excès d...
2      La consommation d’alpha-cyclodextrine à l’occa...
3      Les acides gras essentiels sont nécessaires à ...
4      L’acide alpha-linolénique (ALA) contribue au m...
5      La consommation d’arabinoxylane à l’occasion d...
                             ...                        
256    Le zinc contribue au maintien d’un taux normal...
257    Le zinc contribue au maintien d’une vision nor...
258    Le zinc contribue au fonctionnement normal du ...
259    Le zinc contribue à protéger les cellules cont...
260    Le zinc joue un rôle dans le processus de divi...
Name: Unnamed: 3, Length: 260, dtype: object

1st sentence: Le charbon actif contribue à réduire l’excès de flatulence après le repas


# Create EasyNMT instance

Loading of pre-trained mbart50 and m2m_100 models

## mbart50

In [27]:
from easynmt import EasyNMT


In [28]:
mbart50_model = EasyNMT('mbart50_m2m')

24.9kB [00:00, 6.71MB/s]                   


In [48]:
print("Language directions:")
print(sorted(list(mbart50_model.lang_pairs)))

Language directions:
['af-ar', 'af-az', 'af-bn', 'af-cs', 'af-de', 'af-en', 'af-es', 'af-et', 'af-fa', 'af-fi', 'af-fr', 'af-gl', 'af-gu', 'af-he', 'af-hi', 'af-hr', 'af-id', 'af-it', 'af-ja', 'af-ka', 'af-kk', 'af-km', 'af-ko', 'af-lt', 'af-lv', 'af-mk', 'af-ml', 'af-mn', 'af-mr', 'af-my', 'af-ne', 'af-nl', 'af-pl', 'af-ps', 'af-pt', 'af-ro', 'af-ru', 'af-si', 'af-sl', 'af-sv', 'af-sw', 'af-ta', 'af-te', 'af-th', 'af-tl', 'af-tr', 'af-uk', 'af-ur', 'af-vi', 'af-xh', 'af-zh', 'ar-af', 'ar-az', 'ar-bn', 'ar-cs', 'ar-de', 'ar-en', 'ar-es', 'ar-et', 'ar-fa', 'ar-fi', 'ar-fr', 'ar-gl', 'ar-gu', 'ar-he', 'ar-hi', 'ar-hr', 'ar-id', 'ar-it', 'ar-ja', 'ar-ka', 'ar-kk', 'ar-km', 'ar-ko', 'ar-lt', 'ar-lv', 'ar-mk', 'ar-ml', 'ar-mn', 'ar-mr', 'ar-my', 'ar-ne', 'ar-nl', 'ar-pl', 'ar-ps', 'ar-pt', 'ar-ro', 'ar-ru', 'ar-si', 'ar-sl', 'ar-sv', 'ar-sw', 'ar-ta', 'ar-te', 'ar-th', 'ar-tl', 'ar-tr', 'ar-uk', 'ar-ur', 'ar-vi', 'ar-xh', 'ar-zh', 'az-af', 'az-ar', 'az-bn', 'az-cs', 'az-de', 'az-en', 'az-es

The mBART50 model does not support Hungarian,therefore, only French, German, Polish and Romanian were considered in the implementation and evaluation of the assessment model.

### Evaluate translation

In [ ]:
# evaluate translation
def evaluate_model(a, b):
    # calculate BLEU score
    bleu = sacrebleu.corpus_bleu(a, [b])
    print('BLEU SCORE:')
    print('BLEU: %f' % bleu.score)


## Translation from English

In [52]:
ori = [] # Original English sentences
for line in df.iloc[1:,1]:
  line = line.strip()
  ori.append(line)

print("1st original sentence:", ori[0])

1st original sentence: Activated charcoal contributes to reducing excessive flatulence after eating


### French


In [58]:
fr_preds = mbart50_model.translate(ori, target_lang='fr') # Translation

/usr/local/lib/python3.7/dist-packages/transformers/generation_utils.py:1207: UserWarning: Neither `max_length` nor `max_new_tokens` have been set, `max_length` will default to 200 (`self.config.max_length`). Controlling `max_length` via the config is deprecated and `max_length` will be removed from the config in v5 of Transformers -- we recommend using `max_new_tokens` to control the maximum length of the generation.
  UserWarning,


In [59]:
print("1st predicted sentence:", fr_preds[0])

1st predicted sentence: Le charbon activé contribue à réduire la flatulence excessive après l'alimentation


In [61]:
fr_refs = [] # Original French sentences
for line in df.iloc[1:,3]:
  line = line.strip()
  fr_refs.append(line)

print("1st sentence:", fr_refs[0])

1st sentence: Le charbon actif contribue à réduire l’excès de flatulence après le repas


In [63]:
evaluate_model(fr_preds,fr_refs)# Evaluation

BLEU SCORE:
BLEU: 42.655319


### German

In [64]:
de_preds = mbart50_model.translate(ori, target_lang='de') # Translation

In [65]:
print("1st predicted sentence:", de_preds[0])

1st predicted sentence: Aktivierte Kohle trägt zur Reduzierung der übermäßigen Flatulenz nach dem Essen bei


In [66]:
de_refs = [] # Original German sentences
for line in df.iloc[1:,5]:
  line = line.strip()
  de_refs.append(line)

print("1st sentence:", de_refs[0])

1st sentence: Aktivkohle trägt zur Verringerung übermäßiger Blähungen nach dem Essen bei


In [67]:
evaluate_model(de_preds,de_refs) # Evaluation

BLEU SCORE:
BLEU: 27.331959


### Polish

In [68]:
pl_preds = mbart50_model.translate(ori, target_lang='pl') # Translation

In [69]:
print("1st predicted sentence:", pl_preds[0])

1st predicted sentence: Aktywowane węgla przyczynia się do zmniejszenia nadmiernego płomienia po jedzeniu


In [72]:
pl_refs = [] # Original Polish sentences
for line in df.iloc[1:,7]:
  line = line.strip()
  pl_refs.append(line)

print("1st sentence:", pl_refs[0])

1st sentence: Węgiel aktywowany pomaga zmniejszyć występujące po jedzeniu nadmierne wzdęcia


In [73]:
evaluate_model(pl_preds,pl_refs) # Evaluation

BLEU SCORE:
BLEU: 22.977065


### Romanian

In [74]:
ro_preds = mbart50_model.translate(ori, target_lang='ro') # Translation

In [75]:
print("1st predicted sentence:", ro_preds[0])

1st predicted sentence: Cărbunele activ contribuie la reducerea flatulenţei excesive după mâncare


In [76]:
ro_refs = [] # Original Romanian sentences
for line in df.iloc[1:,9]:
  line = line.strip()
  ro_refs.append(line)

print("1st sentence:", ro_refs[0])

1st sentence: Cărbunele activ contribuie la reducerea flatulenței după consumul de alimente


In [77]:
evaluate_model(ro_preds,ro_refs) # Evaluation

BLEU SCORE:
BLEU: 25.845438


### 1 to Many


In [80]:
predslist = fr_preds+ de_preds+ pl_preds +ro_preds

refslist = fr_refs+ de_refs+ pl_refs +ro_refs

In [81]:
evaluate_model(predslist,refslist) # Evaluation

BLEU SCORE:
BLEU: 31.017612


## Translation to English

In [87]:
en_refs = [] # Reference English sentences
for line in df.iloc[1:,1]:
  line = line.strip()
  en_refs.append(line)

print("1st sentence:", en_refs[0])

1st sentence: Activated charcoal contributes to reducing excessive flatulence after eating


### French

In [89]:
fr_ori = [] # Original French sentences
for line in df.iloc[1:,3]:
  line = line.strip()
  fr_ori.append(line)

print("1st sentence:", fr_ori[0])

1st sentence: Le charbon actif contribue à réduire l’excès de flatulence après le repas


In [90]:
fr_preds = mbart50_model.translate(fr_ori, target_lang='en') # Translation

/usr/local/lib/python3.7/dist-packages/transformers/generation_utils.py:1207: UserWarning: Neither `max_length` nor `max_new_tokens` have been set, `max_length` will default to 200 (`self.config.max_length`). Controlling `max_length` via the config is deprecated and `max_length` will be removed from the config in v5 of Transformers -- we recommend using `max_new_tokens` to control the maximum length of the generation.
  UserWarning,


In [91]:
print("1st predicted sentence:", fr_preds[0])

1st predicted sentence: Active coal helps reduce excess flatulence after meals


In [92]:
evaluate_model(fr_preds,en_refs)# Evaluation

BLEU SCORE:
BLEU: 37.243763


### German

In [93]:
de_ori = [] # Original German sentences
for line in df.iloc[1:,5]:
  line = line.strip()
  de_ori.append(line)

print("1st sentence:", de_ori[0])

1st sentence: Aktivkohle trägt zur Verringerung übermäßiger Blähungen nach dem Essen bei


In [94]:
de_preds = mbart50_model.translate(de_ori, target_lang='en') # Translation

In [95]:
print("1st predicted sentence:", de_preds[0])

1st predicted sentence: Active carbon contributes to the reduction of excessive bleeding after eating


In [96]:
evaluate_model(de_preds,en_refs)# Evaluation

BLEU SCORE:
BLEU: 40.050156


### Polish

In [97]:
pl_ori = [] # Original Polish sentences
for line in df.iloc[1:,7]:
  line = line.strip()
  pl_ori.append(line)

print("1st sentence:", pl_ori[0])

1st sentence: Węgiel aktywowany pomaga zmniejszyć występujące po jedzeniu nadmierne wzdęcia


In [98]:
pl_preds = mbart50_model.translate(pl_ori, target_lang='en') # Translation

In [99]:
print("1st predicted sentence:", pl_preds[0])

1st predicted sentence: Activated carbon helps to reduce overeating after eating


In [100]:
evaluate_model(pl_preds,en_refs)# Evaluation

BLEU SCORE:
BLEU: 22.337401


### Romanian

In [101]:
ro_ori = [] # Original Romanian sentences
for line in df.iloc[1:,9]:
  line = line.strip()
  ro_ori.append(line)

print("1st sentence:", ro_ori[0])

1st sentence: Cărbunele activ contribuie la reducerea flatulenței după consumul de alimente


In [102]:
ro_preds = mbart50_model.translate(ro_ori, target_lang='en') # Translation

In [103]:
print("1st predicted sentence:", ro_preds[0])

1st predicted sentence: Active coal helps reduce post-food flatulence


In [104]:
evaluate_model(ro_preds,en_refs)# Evaluation

BLEU SCORE:
BLEU: 26.964747


### Many to 1

In [105]:
predslist = fr_preds+ de_preds+ pl_preds +ro_preds

refslist = en_refs+ en_refs+ en_refs + en_refs

In [106]:
evaluate_model(predslist,refslist) # Evaluation

BLEU SCORE:
BLEU: 31.884663


## M2M-100

In [ ]:
m2m100_model = EasyNMT('m2m_100_418M')

89.9kB [00:00, 27.5MB/s]                   


In [107]:
print("Language directions:")
print(sorted(list(m2m100_model.lang_pairs)))

Language directions:
['af-am', 'af-ar', 'af-ast', 'af-az', 'af-ba', 'af-be', 'af-bg', 'af-bn', 'af-br', 'af-bs', 'af-ca', 'af-ceb', 'af-cs', 'af-cy', 'af-da', 'af-de', 'af-el', 'af-en', 'af-es', 'af-et', 'af-fa', 'af-ff', 'af-fi', 'af-fr', 'af-fy', 'af-ga', 'af-gd', 'af-gl', 'af-gu', 'af-ha', 'af-he', 'af-hi', 'af-hr', 'af-ht', 'af-hu', 'af-hy', 'af-id', 'af-ig', 'af-ilo', 'af-is', 'af-it', 'af-ja', 'af-jv', 'af-ka', 'af-kk', 'af-km', 'af-kn', 'af-ko', 'af-lb', 'af-lg', 'af-ln', 'af-lo', 'af-lt', 'af-lv', 'af-mg', 'af-mk', 'af-ml', 'af-mn', 'af-mr', 'af-ms', 'af-my', 'af-ne', 'af-nl', 'af-no', 'af-ns', 'af-oc', 'af-or', 'af-pa', 'af-pl', 'af-ps', 'af-pt', 'af-ro', 'af-ru', 'af-sd', 'af-si', 'af-sk', 'af-sl', 'af-so', 'af-sq', 'af-sr', 'af-ss', 'af-su', 'af-sv', 'af-sw', 'af-ta', 'af-th', 'af-tl', 'af-tn', 'af-tr', 'af-uk', 'af-ur', 'af-uz', 'af-vi', 'af-wo', 'af-xh', 'af-yi', 'af-yo', 'af-zh', 'af-zu', 'am-af', 'am-ar', 'am-ast', 'am-az', 'am-ba', 'am-be', 'am-bg', 'am-bn', 'am-br', 'a

## Translation from English

In [160]:
ori = [] # Original English sentences
for line in df.iloc[1:,1]:
  line = line.strip()
  ori.append(line)

print("1st original sentence:", ori[0])

1st original sentence: Activated charcoal contributes to reducing excessive flatulence after eating


### French


In [109]:
fr_preds = m2m100_model.translate(ori, target_lang='fr') # Translation

/usr/local/lib/python3.7/dist-packages/transformers/generation_utils.py:1207: UserWarning: Neither `max_length` nor `max_new_tokens` have been set, `max_length` will default to 200 (`self.config.max_length`). Controlling `max_length` via the config is deprecated and `max_length` will be removed from the config in v5 of Transformers -- we recommend using `max_new_tokens` to control the maximum length of the generation.
  UserWarning,


In [110]:
print("1st predicted sentence:", fr_preds[0])

1st predicted sentence: Le charbon actif contribue à réduire la flatulence excessive après avoir mangé


In [111]:
fr_refs = [] # Original French sentences
for line in df.iloc[1:,3]:
  line = line.strip()
  fr_refs.append(line)

print("1st sentence:", fr_refs[0])

1st sentence: Le charbon actif contribue à réduire l’excès de flatulence après le repas


In [112]:
evaluate_model(fr_preds,fr_refs)# Evaluation

BLEU SCORE:
BLEU: 41.232536


### German

In [113]:
de_preds = m2m100_model.translate(ori, target_lang='de') # Translation

In [114]:
print("1st predicted sentence:", de_preds[0])

1st predicted sentence: Aktiviertes Kohlenstoff trägt zur Verringerung der übermäßigen Flatulenz nach dem Essen bei


In [115]:
de_refs = [] # Original German sentences
for line in df.iloc[1:,5]:
  line = line.strip()
  de_refs.append(line)

print("1st sentence:", de_refs[0])

1st sentence: Aktivkohle trägt zur Verringerung übermäßiger Blähungen nach dem Essen bei


In [116]:
evaluate_model(de_preds,de_refs) # Evaluation

BLEU SCORE:
BLEU: 29.455375


### Polish

In [117]:
pl_preds = m2m100_model.translate(ori, target_lang='pl') # Translation

In [118]:
print("1st predicted sentence:", pl_preds[0])

1st predicted sentence: Aktywny węgiel przyczynia się do zmniejszenia nadmiernej flatulencji po jedzeniu


In [119]:
pl_refs = [] # Original Polish sentences
for line in df.iloc[1:,7]:
  line = line.strip()
  pl_refs.append(line)

print("1st sentence:", pl_refs[0])

1st sentence: Węgiel aktywowany pomaga zmniejszyć występujące po jedzeniu nadmierne wzdęcia


In [120]:
evaluate_model(pl_preds,pl_refs) # Evaluation

BLEU SCORE:
BLEU: 25.769917


### Romanian

In [121]:
ro_preds = m2m100_model.translate(ori, target_lang='ro') # Translation

In [122]:
print("1st predicted sentence:", ro_preds[0])

1st predicted sentence: Carbohidrați activi contribuie la reducerea flatulenței excesive după masă


In [123]:
ro_refs = [] # Original Romanian sentences
for line in df.iloc[1:,9]:
  line = line.strip()
  ro_refs.append(line)

print("1st sentence:", ro_refs[0])

1st sentence: Cărbunele activ contribuie la reducerea flatulenței după consumul de alimente


In [124]:
evaluate_model(ro_preds,ro_refs) # Evaluation

BLEU SCORE:
BLEU: 47.556719


### Hungarian

In [161]:
hu_preds = m2m100_model.translate(ori, target_lang='hu') # Translation

In [162]:
print("1st predicted sentence:", hu_preds[0])

1st predicted sentence: Az aktivált szén hozzájárul az étkezés után a túlzott flatulencia csökkentéséhez


In [163]:
hu_refs = [] 
for line in df.iloc[1:,11]:
  line = line.strip()
  hu_refs.append(line)

print("1st sentence:", hu_refs[0])

1st sentence: Az aktív szén részt vesz az étkezést követő túlzott bélgázképződés csökkentésében.


In [164]:
evaluate_model(hu_preds,hu_refs) # Evaluation

BLEU SCORE:
BLEU: 25.157857


### 1 to Many


In [129]:
predslist = fr_preds+ de_preds+ pl_preds +ro_preds+ hu_preds

refslist = fr_refs+ de_refs+ pl_refs +ro_refs+ hu_refs

In [130]:
evaluate_model(predslist,refslist) # Evaluation

BLEU SCORE:
BLEU: 36.785290


## Translation to English

In [131]:
en_refs = [] # Reference English sentences
for line in df.iloc[1:,1]:
  line = line.strip()
  en_refs.append(line)

print("1st sentence:", en_refs[0])

1st sentence: Activated charcoal contributes to reducing excessive flatulence after eating


### French

In [132]:
fr_ori = [] # Original French sentences
for line in df.iloc[1:,3]:
  line = line.strip()
  fr_ori.append(line)

print("1st sentence:", fr_ori[0])

1st sentence: Le charbon actif contribue à réduire l’excès de flatulence après le repas


In [133]:
fr_preds = m2m100_model.translate(fr_ori, target_lang='en') # Translation

In [134]:
print("1st predicted sentence:", fr_preds[0])

1st predicted sentence: Active coal helps to reduce excess flatulence after meals


In [135]:
evaluate_model(fr_preds,en_refs)# Evaluation

BLEU SCORE:
BLEU: 36.551156


### German

In [136]:
de_ori = [] # Original German sentences
for line in df.iloc[1:,5]:
  line = line.strip()
  de_ori.append(line)

print("1st sentence:", de_ori[0])

1st sentence: Aktivkohle trägt zur Verringerung übermäßiger Blähungen nach dem Essen bei


In [137]:
de_preds = m2m100_model.translate(de_ori, target_lang='en') # Translation

In [138]:
print("1st predicted sentence:", de_preds[0])

1st predicted sentence: Active coal helps to reduce excessive bleeding after eating


In [139]:
evaluate_model(de_preds,en_refs)# Evaluation

BLEU SCORE:
BLEU: 36.872157


### Polish

In [140]:
pl_ori = [] # Original Polish sentences
for line in df.iloc[1:,7]:
  line = line.strip()
  pl_ori.append(line)

print("1st sentence:", pl_ori[0])

1st sentence: Węgiel aktywowany pomaga zmniejszyć występujące po jedzeniu nadmierne wzdęcia


In [141]:
pl_preds = m2m100_model.translate(pl_ori, target_lang='en') # Translation

In [142]:
print("1st predicted sentence:", pl_preds[0])

1st predicted sentence: Activated Coal helps to reduce excess fatigue after eating


In [143]:
evaluate_model(pl_preds,en_refs)# Evaluation

BLEU SCORE:
BLEU: 19.355512


### Romanian

In [144]:
ro_ori = [] # Original Romanian sentences
for line in df.iloc[1:,9]:
  line = line.strip()
  ro_ori.append(line)

print("1st sentence:", ro_ori[0])

1st sentence: Cărbunele activ contribuie la reducerea flatulenței după consumul de alimente


In [145]:
ro_preds = m2m100_model.translate(ro_ori, target_lang='en') # Translation

In [146]:
print("1st predicted sentence:", ro_preds[0])

1st predicted sentence: Active carbon contributes to reducing flatulence after eating food


In [147]:
evaluate_model(ro_preds,en_refs)# Evaluation

BLEU SCORE:
BLEU: 33.465182


### Hungarian

In [148]:
hu_ori = [] # Original Romanian sentences
for line in df.iloc[1:,11]:
  line = line.strip()
  hu_ori.append(line)

print("1st sentence:", hu_ori[0])

1st sentence: Az aktív szén részt vesz az étkezést követő túlzott bélgázképződés csökkentésében.


In [153]:
hu_preds = m2m100_model.translate(hu_ori, target_lang='en') # Translation

In [154]:
print("1st predicted sentence:", hu_preds[0])

1st predicted sentence: Active carbon contributes to the reduction of excessive intestinal gas formation after a meal.


In [155]:
evaluate_model(hu_preds,en_refs)# Evaluation

BLEU SCORE:
BLEU: 26.076781


### Many to 1

In [156]:
predslist = fr_preds+ de_preds+ pl_preds +ro_preds+ hu_preds

refslist = en_refs+ en_refs+ en_refs + en_refs + en_refs

In [157]:
evaluate_model(predslist,refslist) # Evaluation

BLEU SCORE:
BLEU: 28.988182
